## This notebook will walkthrough the relayer's "business-as-usual" script, which actively checks both chains for packets/acks to relay, and times out any old packets.

## WIP

In [ ]:
#imports

In [1]:
#setup lcd clients, rpc urls, wallets

In [2]:
#protobuf imports

In [3]:
#load ibc client, connection, port, channel information from previous notebooks

Query for queued packets on both chains, and filter for not-yet-processed ones

In [11]:
#query the rpc for queued packets that are "unreceived"

In [ ]:
#additionally, filter out (but persist) packets whose timeouts have passed

Relay valid packets, ie. dispatching the MsgRecvPacket message to the destination chain

In [5]:
#update the client on both chains, and fetch packet proofs

In [8]:
#fabricate message & dispatch via stargate

Query for queued acks on both chains, and filter for not-yet-processed ones

In [ ]:
#query the rpc for queued acks that are "unreceived"

Relay valid acks, ie. dispatching the MsgAcknowledgement message to the source chain

In [9]:
#update the client on both chains, and fetch acknowledge proofs

In [10]:
#fabricate message & dispatch via stargate

Cleanup any timed-out packets

In [12]:
#update the client on both chains

In [13]:
#query for the expected next packet sequence number

In [14]:
#fetch nonexistence proof of the timed-out packet

In [ ]:
#fabricate message & dispatch via stargate